In [ ]:
from pathlib import Path

import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import rasterio

from sentinel_downloader import SentinelDownloader, Season
from sentinel_cities_downloader import SentinelCitiesDownloader
from osm_road_data import OSMRoadData
from osm_road_generator import OSMRoadGenerator

In [ ]:
cities_geojson = Path("data/city_ids_and_bounds.geojson")
if not cities_geojson.exists():
    raise FileNotFoundError(cities_geojson)

cities_gdf = gpd.read_file(cities_geojson)

In [ ]:
cities_gdf

In [ ]:
s2_cities_downloader = SentinelCitiesDownloader(years=[2021], 
                                                seasons=[Season.Spring, Season.Summer, Season.Fall],
                                                use_cache=False,
                                                verbose=True,
                                                force_new_download=True)
s2_cities_downloader.download_all(parallel=False, debug=False)

In [ ]:
osm_road_generator = OSMRoadGenerator()
osm_road_generator.generate_roads_parallel()

In [ ]:
plot_city_id = 10
city_name = cities_gdf[cities_gdf["asset_identifier"] == plot_city_id]["asset_name"].iloc[0]

In [ ]:
with rasterio.open(f"data/sentinel2_images/{plot_city_id}/2021/summer.tif") as ds:
    visual_img = np.moveaxis(ds.read(), 0, -1)
with rasterio.open(f"data/osm_images/{plot_city_id}.tif") as ds:
    road_img = np.moveaxis(ds.read(), 0, -1)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(visual_img)
axes[0].get_yaxis().set_visible(False)
axes[0].get_xaxis().set_visible(False)
axes[1].imshow(road_img)
axes[1].get_yaxis().set_visible(False)
axes[1].get_xaxis().set_visible(False)
plt.suptitle(city_name)
